In [ ]:
import pandas as pd
import sqlite3

# ===== 1. ファイルを読む（TSV, 文字化け前提）=====

# 元のファイル名に合わせてパスを設定
tsv_path = "manga_sample_10000_20250423.csv"  # 中身はTSVだが拡張子はcsvでOK

# 「とりあえず壊さず読む」ため encoding='latin1' + sep='\t'
df = pd.read_csv(tsv_path, sep="\t", encoding="latin1")

print("読み込み完了。行数:", len(df))


# ===== 2. 文字化けを自動修正する関数 =====
# 典型的な「Shift_JIS のバイト列を latin1 として誤解釈した」パターンを想定
#   → いったん latin1 としてバイト化し直し、それを cp932(Shift_JIS互換)で復元

def fix_mojibake(val):
    if pd.isna(val):
        return val
    if not isinstance(val, str):
        return val
    try:
        # latin1 文字列 → 元のバイト列に戻す → cp932(Shift_JIS系)として解釈
        return val.encode("latin1").decode("cp932")
    except UnicodeDecodeError:
        return val  # 直せないものはそのまま返す


# 文字化けしている可能性が高い列だけを対象にする
columns_to_fix = [
    "title",
    "author",
    "publisher",
    "magazine",
    "summary",
    "abbreviation",
    "awards",
    "first_animated_news",
    "first_dramatized_news",
]

for col in columns_to_fix:
    if col in df.columns:
        df[col] = df[col].map(fix_mojibake)

# ちゃんと直っているか、ざっと確認
print(df[["title", "author"]].head())


# ===== 3. SQLite に保存する =====

db_path = "manga.db"
table_name = "manga_works"

# ここでテーブルを自分で定義しても良いし、
# to_sql に任せても良いです

# 3-1. （オプション）型を指定してテーブルを事前作成したい場合
schema_sql = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    yyyymm TEXT,
    title TEXT,
    author TEXT,
    publisher TEXT,
    magazine TEXT,
    summary TEXT,
    release_date TEXT,
    status TEXT,
    volume INTEGER,
    abbreviation TEXT,
    awards TEXT,
    score REAL,
    myanimelist_standard_score REAL,
    myanimelist_standard_scored_cnt INTEGER,
    comiccmoa_standard_score REAL,
    comiccmoa_standard_scored_cnt INTEGER,
    comiccmoa_score REAL,
    comiccmoa_scored_cnt INTEGER,
    manga_okoku_score REAL,
    manga_okoku_scored_cnt INTEGER,
    metyacomic_score REAL,
    metyacomic_scored_cnt INTEGER,
    manba_score REAL,
    manba_scored_cnt INTEGER,
    estimated_monthly_search_volume INTEGER,
    wiki_pv INTEGER,
    first_animated_news TEXT,
    first_animated_news_url TEXT,
    first_animated_news_yyyymmdd TEXT,
    first_dramatized_news TEXT,
    first_dramatized_news_url TEXT,
    first_dramatized_news_yyyymmdd TEXT
);
"""

conn = sqlite3.connect(db_path)
cur = conn.cursor()
cur.execute(schema_sql)
conn.commit()

# 3-2. pandas から一括で投入
# 既存データを上書きしたい場合は if_exists="replace"
# 既存テーブルに追加したい場合は if_exists="append"
df.to_sql(table_name, conn, if_exists="replace", index=False)

conn.close()

print("SQLite への書き込み完了:", db_path, "テーブル:", table_name)
